# REST API
<img src='pics/0.png' />

## Request objects

<img src='pics/Screenshot from 2018-03-07 15-38-28.png' />

REST framework introduces a Request object that extends the regular HttpRequest, and provides more flexible request parsing. The core functionality of the Request object is the request.data attribute, which is similar to request.POST, but more useful for working with Web APIs.

In [ ]:
request.POST  # Only handles form data.  Only works for 'POST' method.
request.data  # Handles arbitrary data.  Works for 'POST', 'PUT' and 'PATCH' methods.

## Response objects

REST framework also introduces a Response object, which is a type of TemplateResponse that takes unrendered content and uses content negotiation to determine the correct content type to return to the client.

In [ ]:
return Response(data)  # Renders to content type as requested by the client.

# Request parsing

### .data

request.data returns the parsed content of the request body. This is similar to the standard request.POST and request.FILES attributes except that:

> It includes all parsed content, including file and non-file inputs.

> It supports parsing the content of HTTP methods other than POST, meaning that you can access the content of PUT and PATCH requests.

> It supports REST framework's flexible request parsing, rather than just supporting form data. For example you can handle incoming JSON data in the same way that you handle incoming form data.


### Authentication

REST framework provides flexible, per-request authentication, that gives you the ability to:

>Use different authentication policies for different parts of your API.

>Support the use of multiple authentication policies.

>Provide both user and token information associated with the incoming request.


### .user
request.user typically returns an instance of django.contrib.auth.models.User, although the behavior depends on the authentication policy being used.

If the request is unauthenticated the default value of request.user is an instance of django.contrib.auth.models.AnonymousUser.

### .auth

request.auth returns any additional authentication context. The exact behavior of request.auth depends on the authentication policy being used, but it may typically be an instance of the token that the request was authenticated against.

If the request is unauthenticated, or if no additional context is present, the default value of request.auth is None.

### Status codes

Using numeric HTTP status codes in your views doesn't always make for obvious reading, and it's easy to not notice if you get an error code wrong. REST framework provides more explicit identifiers for each status code, such as HTTP_400_BAD_REQUEST in the status module. It's a good idea to use these throughout rather than using numeric identifiers.

# Creating responses

## Response()
>Signature: Response(data, status=None, template_name=None, headers=None, content_type=None)

Unlike regular HttpResponse objects, you do not instantiate Response objects with rendered content. Instead you pass in unrendered data, which may consist of any Python primitives.

The renderers used by the Response class cannot natively handle complex datatypes such as Django model instances, so you need to serialize the data into primitive datatypes before creating the Response object.

You can use REST framework's Serializer classes to perform this data serialization, or use your own custom serialization.

Arguments:

>data: The serialized data for the response.

>status: A status code for the response. Defaults to 200. See also status codes.

>template_name: A template name to use if HTMLRenderer is selected.

>headers: A dictionary of HTTP headers to use in the response.

>content_type: The content type of the response. Typically, this will be set automatically by the renderer as 
determined by content negotiation, but there may be some cases where you need to specify the content type explicitly.

# views

## Wrapping API views

REST framework provides two wrappers you can use to write API views.

1. The @api_view decorator for working with function based views.

2. The APIView class for working with class-based views.

These wrappers provide a few bits of functionality such as making sure you receive Request instances in your view, and adding context to Response objects so that content negotiation can be performed.

The wrappers also provide behaviour such as returning 405 Method Not Allowed responses when appropriate, and handling any ParseError exception that occurs when accessing request.data with malformed input.

## Function Based Views


REST framework also allows you to work with regular function based views. It provides a set of simple decorators that wrap your function based views to ensure they receive an instance of Request (rather than the usual Django HttpRequest) and allows them to return a Response (instead of a Django HttpResponse), and allow you to configure how the request is processed.

### @api_view

>Signature: @api_view(http_method_names=['GET'])

The core of this functionality is the api_view decorator, which takes a list of HTTP methods that your view should respond to. For example, this is how you would write a very simple view that just manually returns some data:

In [ ]:
from rest_framework.decorators import api_view

@api_view()
def hello_world(request):
    return Response({"message": "Hello, world!"})

This view will use the default renderers, parsers, authentication classes etc specified in the settings.

By default only GET methods will be accepted. Other methods will respond with "405 Method Not Allowed". To alter this behaviour, specify which methods the view allows, like so:

In [ ]:
@api_view(['GET', 'POST'])
def hello_world(request):
    if request.method == 'POST':
        return Response({"message": "Got some data!", "data": request.data})
    return Response({"message": "Hello, world!"})

## Class-based Views


REST framework provides an APIView class, which subclasses Django's View class.

APIView classes are different from regular View classes in the following ways:

>Requests passed to the handler methods will be REST framework's Request instances, not Django's HttpRequest instances.

>Handler methods may return REST framework's Response, instead of Django's HttpResponse. The view will manage content negotiation and setting the correct renderer on the response.

>Any APIException exceptions will be caught and mediated into appropriate responses.

>Incoming requests will be authenticated and appropriate permission and/or throttle checks will be run before dispatching the request to the handler method.

Using the APIView class is pretty much the same as using a regular View class, as usual, the incoming request is dispatched to an appropriate handler method such as .get() or .post(). Additionally, a number of attributes may be set on the class that control various aspects of the API policy.



In [ ]:
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import authentication, permissions
from django.contrib.auth.models import User

class ListUsers(APIView):
    """
    View to list all users in the system.

    * Requires token authentication.
    * Only admin users are able to access this view.
    """
    authentication_classes = (authentication.TokenAuthentication,)
    permission_classes = (permissions.IsAdminUser,)

    def get(self, request, format=None):
        """
        Return a list of all users.
        """
        usernames = [user.username for user in User.objects.all()]
        return Response(usernames)

## API policy attributes

.renderer_classes

.parser_classes

.authentication_classes

.throttle_classes

.permission_classes

.content_negotiation_class

## API policy instantiation methods
The following methods are used by REST framework to instantiate the various pluggable API policies. You won't typically need to override these methods.

.get_renderers(self)

.get_parsers(self)

.get_authenticators(self)

.get_throttles(self)

.get_permissions(self)

.get_content_negotiator(self)

.get_exception_handler(self)

## API policy implementation methods
The following methods are called before dispatching to the handler method.

.check_permissions(self, request)

.check_throttles(self, request)

.perform_content_negotiation(self, request, force=False)

# Generic views

One of the key benefits of class-based views is the way they allow you to compose bits of reusable behavior. REST framework takes advantage of this by providing a number of pre-built views that provide for commonly used patterns.

The generic views provided by REST framework allow you to quickly build API views that map closely to your database models.

If the generic views don't suit the needs of your API, you can drop down to using the regular APIView class, or reuse the mixins and base classes used by the generic views to compose your own set of reusable generic views.

Typically when using the generic views, you'll override the view, and set several class attributes.

In [ ]:
from django.contrib.auth.models import User
from myapp.serializers import UserSerializer
from rest_framework import generics
from rest_framework.permissions import IsAdminUser

class UserList(generics.ListCreateAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    permission_classes = (IsAdminUser,)

For more complex cases you might also want to override various methods on the view class. 

In [ ]:
class UserList(generics.ListCreateAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    permission_classes = (IsAdminUser,)

    def list(self, request):
        # Note the use of `get_queryset()` instead of `self.queryset`
        queryset = self.get_queryset()
        serializer = UserSerializer(queryset, many=True)
        return Response(serializer.data)

For very simple cases you might want to pass through any class attributes using the .as_view() method. For example, your URLconf might include something like the following entry:

In [ ]:
url(
    r'^/users/',
    ListCreateAPIView.as_view(queryset=User.objects.all(), serializer_class=UserSerializer),
    name='user-list'
)

# GenericAPIView

## Attributes

### Basic settings:

The following attributes control the basic view behavior.

>queryset - The queryset that should be used for returning objects from this view. Typically, you must either set this attribute, or override the get_queryset() method. If you are overriding a view method, it is important that you call get_queryset() instead of accessing this property directly, as queryset will get evaluated once, and those results will be cached for all subsequent requests.

>serializer_class - The serializer class that should be used for validating and deserializing input, and for serializing output. Typically, you must either set this attribute, or override the get_serializer_class() method.

>lookup_field - The model field that should be used to for performing object lookup of individual model instances. Defaults to 'pk'. Note that when using hyperlinked APIs you'll need to ensure that both the API views and the serializer classes set the lookup fields if you need to use a custom value.

>lookup_url_kwarg - The URL keyword argument that should be used for object lookup. The URL conf should include a keyword argument corresponding to this value. If unset this defaults to using the same value as lookup_field.

### Pagination:

The following attributes are used to control pagination when used with list views.

pagination_class - The pagination class that should be used when paginating list results. Defaults to the same value as the DEFAULT_PAGINATION_CLASS setting, which is 'rest_framework.pagination.PageNumberPagination'. Setting pagination_class=None will disable pagination on this view.

### Filtering:

filter_backends - A list of filter backend classes that should be used for filtering the queryset. Defaults to the same value as the DEFAULT_FILTER_BACKENDS setting.

## Methods


### get_queryset(self)
Returns the queryset that should be used for list views, and that should be used as the base for lookups in detail views. Defaults to returning the queryset specified by the queryset attribute.

This method should always be used rather than accessing self.queryset directly, as self.queryset gets evaluated only once, and those results are cached for all subsequent requests.

May be overridden to provide dynamic behavior, such as returning a queryset, that is specific to the user making the request.

In [ ]:
def get_queryset(self):
    user = self.request.user
    return user.accounts.all()

### get_object(self)
Returns an object instance that should be used for detail views. Defaults to using the lookup_field parameter to filter the base queryset.

May be overridden to provide more complex behavior, such as object lookups based on more than one URL kwarg.

In [ ]:
def get_object(self):
    queryset = self.get_queryset()
    filter = {}
    for field in self.multiple_lookup_fields:
        filter[field] = self.kwargs[field]

    obj = get_object_or_404(queryset, **filter)
    self.check_object_permissions(self.request, obj)
    return obj

### filter_queryset(self, queryset)

Given a queryset, filter it with whichever filter backends are in use, returning a new queryset.

In [ ]:
def filter_queryset(self, queryset):
    filter_backends = (CategoryFilter,)

    if 'geo_route' in self.request.query_params:
        filter_backends = (GeoRouteFilter, CategoryFilter)
    elif 'geo_point' in self.request.query_params:
        filter_backends = (GeoPointFilter, CategoryFilter)

    for backend in list(filter_backends):
        queryset = backend().filter_queryset(self.request, queryset, view=self)


### get_serializer_class(self)
Returns the class that should be used for the serializer. Defaults to returning the serializer_class attribute.

May be overridden to provide dynamic behavior, such as using different serializers for read and write operations, or providing different serializers to different types of users.

In [ ]:
def get_serializer_class(self):
    if self.request.user.is_staff:
        return FullAccountSerializer
    return BasicAccountSerializer

### Save and deletion hooks:

The following methods are provided by the mixin classes, and provide easy overriding of the object save or deletion behavior.

1. perform_create(self, serializer) - Called by CreateModelMixin when saving a new object instance.
2. perform_update(self, serializer) - Called by UpdateModelMixin when saving an existing object instance.
3. perform_destroy(self, instance) - Called by DestroyModelMixin when deleting an object instance.

These hooks are particularly useful for setting attributes that are implicit in the request, but are not part of the request data. For instance, you might set an attribute on the object based on the request user, or based on a URL keyword argument.

In [ ]:
def perform_create(self, serializer):
    serializer.save(user=self.request.user)

These override points are also particularly useful for adding behavior that occurs before or after saving an object, such as emailing a confirmation, or logging the update.

In [ ]:
def perform_update(self, serializer):
    instance = serializer.save()
    send_email_confirmation(user=self.request.user, modified=instance)

You can also use these hooks to provide additional validation, by raising a ValidationError(). This can be useful if you need some validation logic to apply at the point of database save. For example:



In [ ]:
def perform_create(self, serializer):
    queryset = SignupRequest.objects.filter(user=self.request.user)
    if queryset.exists():
        raise ValidationError('You have already signed up')
    serializer.save(user=self.request.user)

# Mixins

The mixin classes provide the actions that are used to provide the basic view behavior. Note that the mixin classes provide action methods rather than defining the handler methods, such as .get() and .post(), directly. This allows for more flexible composition of behavior.

The mixin classes can be imported from rest_framework.mixins.

### ListModelMixin
Provides a .list(request, *args, **kwargs) method, that implements listing a queryset.

If the queryset is populated, this returns a 200 OK response, with a serialized representation of the queryset as the body of the response. The response data may optionally be paginated.

### CreateModelMixin
Provides a .create(request, *args, **kwargs) method, that implements creating and saving a new model instance.

If an object is created this returns a 201 Created response, with a serialized representation of the object as the body of the response. If the representation contains a key named url, then the Location header of the response will be populated with that value.

If the request data provided for creating the object was invalid, a 400 Bad Request response will be returned, with the error details as the body of the response.

### RetrieveModelMixin
Provides a .retrieve(request, *args, **kwargs) method, that implements returning an existing model instance in a response.

If an object can be retrieved this returns a 200 OK response, with a serialized representation of the object as the body of the response. Otherwise it will return a 404 Not Found.

### UpdateModelMixin
Provides a .update(request, *args, **kwargs) method, that implements updating and saving an existing model instance.

Also provides a .partial_update(request, *args, **kwargs) method, which is similar to the update method, except that all fields for the update will be optional. This allows support for HTTP PATCH requests.

If an object is updated this returns a 200 OK response, with a serialized representation of the object as the body of the response.

If the request data provided for updating the object was invalid, a 400 Bad Request response will be returned, with the error details as the body of the response.

### DestroyModelMixin
Provides a .destroy(request, *args, **kwargs) method, that implements deletion of an existing model instance.

If an object is deleted this returns a 204 No Content response, otherwise it will return a 404 Not Found.

# Concrete View Classes
The following classes are the concrete generic views. If you're using generic views this is normally the level you'll be working at unless you need heavily customized behavior.

>The view classes can be imported from rest_framework.generics.

### CreateAPIView
Used for create-only endpoints.

Provides a post method handler.

>Extends: GenericAPIView, CreateModelMixin

### ListAPIView
Used for read-only endpoints to represent a collection of model instances.

Provides a get method handler.

>Extends: GenericAPIView, ListModelMixin

### RetrieveAPIView
Used for read-only endpoints to represent a single model instance.

Provides a get method handler.

>Extends: GenericAPIView, RetrieveModelMixin

### DestroyAPIView
Used for delete-only endpoints for a single model instance.

Provides a delete method handler.

>Extends: GenericAPIView, DestroyModelMixin

### UpdateAPIView
Used for update-only endpoints for a single model instance.

Provides put and patch method handlers.

>Extends: GenericAPIView, UpdateModelMixin

### ListCreateAPIView
Used for read-write endpoints to represent a collection of model instances.

Provides get and post method handlers.

>Extends: GenericAPIView, ListModelMixin, CreateModelMixin

### RetrieveUpdateAPIView
Used for read or update endpoints to represent a single model instance.

Provides get, put and patch method handlers.

>Extends: GenericAPIView, RetrieveModelMixin, UpdateModelMixin

### RetrieveDestroyAPIView
Used for read or delete endpoints to represent a single model instance.

Provides get and delete method handlers.

>Extends: GenericAPIView, RetrieveModelMixin, DestroyModelMixin

### RetrieveUpdateDestroyAPIView
Used for read-write-delete endpoints to represent a single model instance.

Provides get, put, patch and delete method handlers.

>Extends: GenericAPIView, RetrieveModelMixin, UpdateModelMixin, DestroyModelMixin

# Author api

In [ ]:
from django.http import HttpResponse

from rest_framework import generics
from rest_framework import status
from rest_framework.decorators import api_view
from rest_framework.response import Response

from . import serializers
from my_app import models


@api_view(['GET', 'POST'])
def author_list(request):
    """
    List all code Authors, or create a new author.
    """
    if request.method == 'GET':
        authors = models.Author.objects.all()
        serializer = serializers.AuthorSerializer(authors, many=True)
        return Response(serializer.data)

    elif request.method == 'POST':
        serializer = serializers.AuthorSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


@api_view(['GET', 'PUT', 'DELETE'])
def author_detail(request, pk):
    """
    Retrieve, update or delete a code author.
    """
    try:
        author = models.Author.objects.get(pk=pk)
    except models.Author.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = serializers.AuthorSerializer(author)
        return Response(serializer.data)

    elif request.method == 'PUT':
        serializer = serializers.AuthorSerializer(author, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        author.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)


## urls

In [ ]:
from django.conf.urls import url

from rest_framework_swagger.views import get_swagger_view

from . import views

swagger = get_swagger_view(title='Public API')

urlpatterns = [

    url(
        r'^author/$',
        view=views.author_list,
        name='author-list-api-view'
    ),

    url(
        r'^author/(?P<pk>[0-9]+)$',
        views.author_detail,
        name='author_detail-api-view'
    ),

    url(r'^swagger/$', view=swagger, name='swagger'),
]


<img src='pics/Screenshot from 2018-03-07 13-51-26.png' />

## APIView

In [ ]:
from django.http import HttpResponse, Http404

from rest_framework import status, generics
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework.views import APIView

from . import serializers
from my_app import models

class AuthorList(generics.GenericAPIView):
    """
    List all authors, or create a new Author.
    """

    serializer_class = serializers.AuthorSerializer
    
    def get(self, request, format=None):
        authors = models.Author.objects.all()
        serializer = serializers.AuthorSerializer(authors, many=True)
        return Response(serializer.data)

    def post(self, request, format=None):
        serializer = serializers.AuthorSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


class AuthorDetail(generics.GenericAPIView):
    """
    Retrieve, update or delete a author instance.
    """

    serializer_class = serializers.AuthorSerializer

    def get_object(self, pk):
        try:
            return models.Author.objects.get(pk=pk)
        except models.Author.DoesNotExist:
            raise Http404

    def get(self, request, pk, format=None):
        author = self.get_object(pk)
        serializer = serializers.AuthorSerializer(author)
        return Response(serializer.data)

    def put(self, request, pk, format=None):
        snippet = self.get_object(pk)
        serializer = serializers.AuthorSerializer(snippet, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk, format=None):
        author = self.get_object(pk)
        author.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)


## urls

In [ ]:
from django.conf.urls import url

from rest_framework_swagger.views import get_swagger_view

from . import views

swagger = get_swagger_view(title='Public API')

urlpatterns = [

    url(
        r'^author/$',
        view=views.AuthorList.as_view(),
        name='author-list-api-view'
    ),

    url(
        r'^author/(?P<pk>[0-9]+)$',
        views.AuthorDetail.as_view(),
        name='author_detail-api-view'
    ),

    url(r'^swagger/$', view=swagger, name='swagger'),
]


<img src='pics/Screenshot from 2018-03-07 14-32-44.png' />

# views

In [ ]:
class AuthorList(generics.ListAPIView):
    serializer_class = serializers.AuthorSerializer

    def get_queryset(self, ):
        return models.Author.objects.all()


class AuthorDetail(generics.RetrieveAPIView):
    serializer_class = serializers.AuthorSerializer
    queryset = models.Author.objects.all()

# serializer

In [ ]:

class AuthorSerializer(serializers.ModelSerializer):
    class Meta:
        model = models.Author
        fields = ['id', 'first_name', 'last_name',
                  'date_of_birth', 'date_of_death',
        ]

# url

In [ ]:
url(
    r'^author/$',
    view=views.AuthorList.as_view(),
    name='author-list-api-view'
),

url(
    r'^author/(?P<pk>[0-9]+)/$',
    views.AuthorDetail.as_view(),
    name='author-detail-api-view'
),


<img src='pics/Screenshot from 2018-04-11 10-54-58.png' />

# views

In [ ]:
class AuthorList(generics.ListCreateAPIView):
    serializer_class = serializers.AuthorSerializer

    def get_queryset(self, ):
        return models.Author.objects.all()


class AuthorDetail(generics.RetrieveUpdateDestroyAPIView):
    serializer_class = serializers.AuthorSerializer
    queryset = models.Author.objects.all()


<img src='pics/Screenshot from 2018-04-11 11-02-59.png' />

# views

In [ ]:
class AuthorList(
            generics.GenericAPIView,
            generics.mixins.ListModelMixin,
            generics.mixins.CreateModelMixin,):

    serializer_class = serializers.AuthorSerializer

    def get_queryset(self, ):
        return models.Author.objects.all()

    def get(self, request, *args, **kwargs):
        return self.list(request, *args, **kwargs)

    def post(self, request, *args, **kwargs):
        return self.create(request, *args, **kwargs)


class AuthorDetail(
        generics.GenericAPIView,
        generics.mixins.DestroyModelMixin,
        generics.mixins.RetrieveModelMixin,
        generics.mixins.UpdateModelMixin):

    serializer_class = serializers.AuthorSerializer
    queryset = models.Author.objects.all()

    def get(self, request, *args, **kwargs):
        return self.retrieve(request, *args, **kwargs)

    def put(self, request, *args, **kwargs):
        return self.update(request, *args, **kwargs)

    def delete(self, request, *args, **kwargs):
        return self.destroy(request, *args, **kwargs)

<img src='pics/Screenshot from 2018-04-11 11-24-29.png' />
<img src='pics/1922762_1.jpg' />